In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [1]:
# Create a complete list of urls linking to all TREC conference overviews in the year 2000 - 2022

link_list = []
for i in range(9,32):
    link_list.append(f"https://pages.nist.gov/trec-browser/trec{i}/overview/")


In [3]:
link_list

['https://pages.nist.gov/trec-browser/trec9/overview/',
 'https://pages.nist.gov/trec-browser/trec10/overview/',
 'https://pages.nist.gov/trec-browser/trec11/overview/',
 'https://pages.nist.gov/trec-browser/trec12/overview/',
 'https://pages.nist.gov/trec-browser/trec13/overview/',
 'https://pages.nist.gov/trec-browser/trec14/overview/',
 'https://pages.nist.gov/trec-browser/trec15/overview/',
 'https://pages.nist.gov/trec-browser/trec16/overview/',
 'https://pages.nist.gov/trec-browser/trec17/overview/',
 'https://pages.nist.gov/trec-browser/trec18/overview/',
 'https://pages.nist.gov/trec-browser/trec19/overview/',
 'https://pages.nist.gov/trec-browser/trec20/overview/',
 'https://pages.nist.gov/trec-browser/trec21/overview/',
 'https://pages.nist.gov/trec-browser/trec22/overview/',
 'https://pages.nist.gov/trec-browser/trec23/overview/',
 'https://pages.nist.gov/trec-browser/trec24/overview/',
 'https://pages.nist.gov/trec-browser/trec25/overview/',
 'https://pages.nist.gov/trec-br

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Initialize the data structure for the DataFrame
data = []

# Iterate through the list of links
for i in link_list:
    response = requests.get(i)
    pattern = re.compile(r'trec\d{1,2}')
    match = pattern.search(i)
    
    if match:
        trec_version = match.group()
    else:
        continue  

    link_beginning = f"https://pages.nist.gov/trec-browser/{trec_version}/"
    
    # Parsing the HTML content of the website
    soup = BeautifulSoup(response.text, 'html.parser')

    # Run through all <h2> tags
    for h2 in soup.find_all('h2'):
        # Extract the text of the <h2> tag
        h2_text = h2.get_text(strip=True)[:-1]
        
        # Finde den nächsten <p> Tag nach dem <h2> Tag
        next_p = h2.find_next('p')
        
        if next_p:
            # Find the next <p> tag after the <h2> tag
            a_tags = next_p.find_all('a')
            
            if len(a_tags) >= 2:
                # Extract the href attribute value of the second <a> tag
                second_a_href = a_tags[1].get('href', 'Kein href gefunden')
                page_url = link_beginning + second_a_href[3:]
                
                # Request to the page specified in the href attribute of the second <a> tag
                page_response = requests.get(page_url)
                page_soup = BeautifulSoup(page_response.text, 'html.parser')
                
                # Find all <strong> tags with the text “Paper:”
                strong_tags = page_soup.find_all('strong', string=re.compile('Paper:'))
                
                links_list = []
                
                for strong in strong_tags:
                    next_a = strong.find_next('a')
                    if next_a:
                        links_list.append(next_a.get('href', 'Kein href gefunden'))

                # Add the extracted data to the DataFrame
                data.append({
                    'TREC Version': trec_version,
                    'Track Name': h2_text,
                    'Links': links_list
                })

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

In [6]:
df

,TREC Version,Track Name,Links
0,trec9,Web,[http://trec.nist.gov/pubs/trec9/papers/web9.p...
1,trec9,Spoken Document Retrieval,[http://trec.nist.gov/pubs/trec9/sdrt9_slides/...
2,trec9,Question Answering,[http://trec.nist.gov/pubs/trec9/papers/qa_ove...
3,trec9,Cross-Language,[http://trec.nist.gov/pubs/trec9/papers/trec9-...
4,trec9,Filtering,[http://trec.nist.gov/pubs/trec9/papers/filter...
...,...,...,...
161,trec31,Deep Learning,[https://trec.nist.gov/pubs/trec31/papers/Over...
162,trec31,Conversational Assistance,[https://trec.nist.gov/pubs/trec31/papers/Over...
163,trec31,Clinical Trials,[https://trec.nist.gov/pubs/trec31/papers/Over...
164,trec31,Fair Ranking,[https://trec.nist.gov/pubs/trec31/papers/Over...


In [41]:
df.to_parquet("../../../data/trec_assign_tracks_to_pdf.parquet")

In [9]:
df_trec = pd.read_parquet("../../../data/metadata_TREC.parquet")

In [11]:
# List of Overpaper titles to assign Overview instead of Track name later

list_overview = ["Overview of the Ninth Text REtrieval Conference (TREC-9).", "Overview of TREC 2001.", "Overview of TREC 2002.", "Overview of TREC 2003.", "Overview of TREC 2004.",
                "Overview of TREC 2005.", "Overview of the TREC 2006.", "Overview of TREC 2007.", "Overview of TREC 2021."]

In [14]:
# Manually curated list of document related Tracks that are missing on the website
additional_list = [['Evaluation'],
 ['Interactive'],
 ['Interactive'],
 ['Web'],
 ['Question Answering'],
 ['Interactive'],
 ['Cross-Language', 'Web', 'Interactive'],
 ['Web'],
 ['Others'],
 ['HARD'],
 ['HARD'],
 ['Genomics'],
 ['Legal'],
 ['Relevance Feedback'],
 ['Clinical Decision Support', 'Microblog'],
 ['Contextual Suggestion'],
 ['Deep Learning'],
 ['Fair Ranking'],
 ['LiveQA'],
 ['Web'],
 ['Decision'],
 ['Microblog'],
 ['Web'],
 ['LiveQA', 'Real-time Summarization'],
 ['Chemical'],
 ['Deep Learning'],
 ['Deep Learning'],
 ['CENTRE'],
 ['Fair Ranking'],
 ['Precision Medicine'],
 ['Incident Streams'],
 ['News'],
 ['Real-time Summarization'],
 ['Real-time Summarization'],
 ['News'],
 ['Session'],
 ['Incident Streams'],
 ['Conversational Assistance'],
 ['Contextual Suggestion'],
 ['Deep Learning'],
 ['Clinical Decision Support'],
 ['Crowdsourcing'],
 ['Complex Answer Retrieval'],
 ['Million Query'],
 ['Fair Ranking'],
 ['Precision Medicine'],
 ['Microblog']]

In [16]:
# Assign missing Labels based on the Dataframe and the two created lists

tracks_rows = []

for i , j in df_trec.iterrows():
    tracks = []
    for k, l in df.iterrows():
        if j["url"] in l["Links"]:
            tracks.append(l["Track Name"])
    if len(tracks) == 0 and j["Title"] in list_overview:
        tracks.append("Overview")
    if len(tracks) == 0:
        tracks = additional_list[counter]
        counter += 1
    tracks_rows.append(tracks) 

2000 http://trec.nist.gov/pubs/trec9/papers/liggett.pdf Query Expansion Seen Through Return Order of Relevant Documents. trec_5
['Evaluation']
2000 http://trec.nist.gov/pubs/trec9/papers/chapman.pdf Passive Feedback Collection--An Attempt to Debunk the Myth of Clickthroughs. trec_24
['Interactive']
2000 http://trec.nist.gov/pubs/trec9/papers/ohsu.pdf Further Analysis of Whether Batch and User Evaluations Give the Same Results with a Question-Answering Task. trec_43
['Interactive']
2000 http://trec.nist.gov/pubs/trec9/papers/kutrec9.pdf Kasetsart University TREC-9 Experiments. trec_68
['Web']
2001 http://trec.nist.gov/pubs/trec10/papers/Trec10NotebookPrager.pdf Use of WordNet Hypernyms for Answering What-Is Questions. trec_100
['Question Answering']
2001 http://trec.nist.gov/pubs/trec10/papers/TREC-2001-04.pdf Important Cognitive Components of Domain-Specific Search Knowledge. trec_142
['Interactive']
2002 http://trec.nist.gov/pubs/trec11/papers/unc-ch.newby.pdf Progress in General-Purp

In [19]:
df_trec["Tracks"] = tracks_rows

In [23]:
df_trec = df_trec[['ID', 'Tracks']]


In [24]:
df_trec.to_parquet("../../../data/Assign_tracks_by_ID_for_TREC.parquet")